In [ ]:
# Load env + connect to SQL Server

In [ ]:
import os
from dotenv import load_dotenv
import pyodbc
import pandas as pd

# Load environment variables
load_dotenv()

server = os.getenv("MSSQL_HOST")
database = os.getenv("MSSQL_DB")
username = os.getenv("MSSQL_USER")
password = os.getenv("MSSQL_PASS")

connection_string = (
    "DRIVER={ODBC Driver 18 for SQL Server};"
    f"SERVER={server};"
    f"DATABASE={database};"
    f"UID={username};"
    f"PWD={password};"
    "Encrypt=yes;"
    "TrustServerCertificate=no;"
)

conn = pyodbc.connect(connection_string)
print("✅ Good to go!")

In [ ]:
# Helper Function
def query(sql):
    df = pd.read_sql(sql, conn)
    return df

In [ ]:
# Simple test
query("SELECT TOP 5 * FROM automations.people;")

In [ ]:
========= SCENARIOS ==========

In [ ]:
# Count people by company
query("""
SELECT c.name AS company_name, COUNT(p.id) AS total_people
FROM automations.people p
JOIN automations.companies c ON p.companyID = c.id
GROUP BY c.name
ORDER BY total_people DESC;
""")


In [ ]:
# People who purchased products
query("""
SELECT 
    p.fullName,
    c.name AS company,
    pr.title AS product,
    cp.value,
    cp.purchaseDate
FROM automations.companyProducts cp
JOIN automations.people p ON cp.personID = p.id
JOIN automations.products pr ON cp.productID = pr.id
JOIN automations.companies c ON cp.companyID = c.id
ORDER BY cp.purchaseDate DESC;
""")


In [ ]:
# Supplier network graph
query("""
SELECT 
    c.name AS company,
    s.name AS supplier,
    pr.title AS product
FROM automations.suppliers sp
JOIN automations.companies c ON sp.companyID = c.id
JOIN automations.companies s ON sp.supplierID = s.id
JOIN automations.products pr ON sp.productID = pr.id
ORDER BY c.name;
""")


In [ ]:
# Company KPIs
query("""
SELECT 
    c.name,
    COUNT(DISTINCT p.id) AS employees,
    COUNT(DISTINCT cp.productID) AS purchased_products,
    SUM(cp.value) AS total_spend
FROM automations.companies c
LEFT JOIN automations.people p ON p.companyID = c.id
LEFT JOIN automations.companyProducts cp ON cp.companyID = c.id
GROUP BY c.name
ORDER BY total_spend DESC;
""")


In [ ]:
# Product Performance
query("""
SELECT 
    pr.title,
    pr.category,
    COUNT(cp.id) AS total_sales,
    SUM(cp.value) AS revenue
FROM automations.products pr
LEFT JOIN automations.companyProducts cp ON cp.productID = pr.id
GROUP BY pr.title, pr.category
ORDER BY revenue DESC;
""")
